In [1]:
from pyspark.sql import SparkSession
import os

data_dir = '../../data/Spark_ML/Natural_Language_Processing/smsspamcollection/'

spark = SparkSession.builder.appName('nlp').getOrCreate()

data = spark.read.csv(os.path.join(data_dir, 'SMSSpamCollection'), inferSchema=True, sep='\t')

data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

data.printSchema()

data.show()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Analyze, clean and prepare data

In [2]:
from pyspark.sql.functions import length, udf
from pyspark.sql.types import IntegerType

custom_length = udf(lambda text:len(text), IntegerType())

data = data.withColumn('length', length(data['text']))
data = data.withColumn('custom_length', custom_length(data['text']))

data.show()

data.groupBy('class').mean().show()

+-----+--------------------+------+-------------+
|class|                text|length|custom_length|
+-----+--------------------+------+-------------+
|  ham|Go until jurong p...|   111|          111|
|  ham|Ok lar... Joking ...|    29|           29|
| spam|Free entry in 2 a...|   155|          155|
|  ham|U dun say so earl...|    49|           49|
|  ham|Nah I don't think...|    61|           61|
| spam|FreeMsg Hey there...|   147|          147|
|  ham|Even my brother i...|    77|           77|
|  ham|As per your reque...|   160|          160|
| spam|WINNER!! As a val...|   157|          157|
| spam|Had your mobile 1...|   154|          154|
|  ham|I'm gonna be home...|   109|          109|
| spam|SIX chances to wi...|   136|          136|
| spam|URGENT! You have ...|   155|          155|
|  ham|I've been searchi...|   196|          196|
|  ham|I HAVE A DATE ON ...|    35|           35|
| spam|XXXMobileMovieClu...|   149|          149|
|  ham|Oh k...i'm watchi...|    26|           26|


### Feature Transformations

In [3]:
from pyspark.ml.feature import StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer, IDF

label_changer = StringIndexer(inputCol='class', outputCol='label')
tokenizer = Tokenizer(inputCol='text', outputCol='tokenized')
sw_remover = StopWordsRemover(inputCol='tokenized', outputCol='sw_removed')
count_vectorizer = CountVectorizer(inputCol='sw_removed', outputCol='count_vectorized')
idf = IDF(inputCol='count_vectorized', outputCol='tf_idf')

In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

assembler = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

### Build the data preparation pipeline

In [5]:
from pyspark.ml import Pipeline

data_pipeline = Pipeline(stages=[label_changer, tokenizer, sw_remover, count_vectorizer, idf, assembler])

data_pipeline_model = data_pipeline.fit(data)

clean_data = data_pipeline_model.transform(data)

## Build the model

In [6]:
from pyspark.ml.classification import NaiveBayes

# using default parameters
model = NaiveBayes()

In [7]:
clean_data = clean_data.select('label', 'features')

clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [9]:
tr_data, te_data = clean_data.randomSplit([0.7, 0.3])

spam_predictor = model.fit(tr_data)

te_results = spam_predictor.transform(te_data)

te_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-835.11615909192...|[1.0,6.3214018485...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-806.98794750745...|[1.0,4.5136633986...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-662.85362239622...|[1.0,5.6374088576...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-888.24293330816...|[1.0,7.6656051068...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1377.5735319731...|[1.0,1.0539775703...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-966.58724458074...|[1.0,1.7913128099...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1535.9019834821...|[2.91083036064313...|       1.0|
|  0.0|(13424,[0,1,31,43...|[-344.95873854503...|[1.0,5.9348054056...|       0.0|
|  0.0|(13424,[0,1,874,1...|[-96.086512396046...|[0.99999997574053...|       0.0|
|  0.0|(13424,[0

## Evaluate the model

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_eval_acc = MulticlassClassificationEvaluator(metricName='accuracy')
multi_eval_f = MulticlassClassificationEvaluator(metricName='f1')

acc = multi_eval_acc.evaluate(te_results)
f_score = multi_eval_f.evaluate(te_results)
print("Accuracy of model at predicting spam was: {}".format(acc))
print("F-score of model at predicting spam was: {}".format(f_score))

Accuracy of model at predicting spam was: 0.9180611656087709
F-score of model at predicting spam was: 0.9247453288157873


In [13]:
from pyspark.sql.functions import count

clean_data.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 4827|
|  1.0|  747|
+-----+-----+

